In [ ]:
import utils

f0_predictor = utils.get_f0_predictor(
    "fcpe",
    sampling_rate=22050,
    hop_length=256,
    device="cpu",
    threshold=0.05,
)

In [ ]:
import torch

# Load the model
model = torch.load("pretrain/fcpe_c_v001.pt", map_location="cpu")
model.keys()

In [ ]:
from modules.cfm.dit import DiT
import torch

dit = DiT(
    in_channels=100 * 2,
    hidden_channels=256,
    out_channels=100,
    filter_channels=256 * 4,
    dropout=0.05,
    n_layers=4,
    n_heads=4,
    kernel_size=3,
    utt_emb_dim=512,
)

# print dit parameter count
print(f"DiT parameter count: {sum(p.numel() for p in dit.parameters())}")

x = torch.randn(1, 100, 128)
x_mask = torch.ones(1, 1, 128)
mu = torch.randn(1, 100, 128)
t = torch.Tensor([0.2])
spks = torch.randn(1, 512)
cond = torch.randn(1, 192, 32)
cond_mask = torch.ones(1, 1, 32)

dit(x, x_mask, mu, t, spks, cond, cond_mask).shape

In [2]:
import torch
from models import SynthesizerTrn

vc_model = SynthesizerTrn(
    spec_channels=128,
    hidden_channels=192,
    filter_channels=768,
    n_heads=2,
    n_layers=6,
    kernel_size=3,
    p_dropout=0.1,
    speaker_embedding=512,
    n_speakers=10,
    ssl_dim=768,
    ppgs_dim=40,
)

c = torch.randn(1, 768, 56)
c_lengths = torch.Tensor([56])
ppgs = torch.randn(1, 40, 56)
spec = torch.randn(1, 128, 56)
f0 = torch.randn(1, 1, 56)
uv = torch.ones(1, 56)
g = torch.randn(1, 512)


# print dit parameter count
print(f"VC model parameter count: {sum(p.numel() for p in vc_model.parameters())}")

# (prior_loss, diff_loss, f0_pred, lf0)
vc_model(c=c, f0=f0, uv=uv, spec=spec, ppgs=ppgs, c_lengths=c_lengths)

Using DiT
VC model parameter count: 51650217


AttributeError: 'tuple' object has no attribute 'shape'

In [ ]:
cond, cond_mask, g = vc_model.compute_conditional_latent([spec], [c_lengths])
g.shape, cond.shape, cond_mask.shape

In [ ]:
deocder_output, _ = vc_model.infer(c=c, f0=f0, uv=uv, spec=spec, ppgs=ppgs)
deocder_output.shape

In [ ]:
from torch import nn
from einops.layers.torch import Rearrange

def temporal_avg_pool(x, mask=None):

    len_ = mask.sum(dim=-1)
    return torch.sum(x * mask, dim=-1) / len_

latents = torch.randn(1, 192, 32)
latents_mask = torch.ones(1, 1, 32)

meanpooled_latents = temporal_avg_pool(latents, latents_mask)
meanpooled_latents.shape

In [ ]:
import ppgs
from glob import glob

wavs_paths = glob("/home/alex/Projekte/TestData/Extended/Dexter/**/*.wav", recursive=True)
ppgs_paths = [path.replace(".wav", ".ppg.pt") for path in wavs_paths]

ppgs.from_files_to_files(wavs_paths, ppgs_paths, gpu=None)


In [ ]:
from glob import glob

wavs_paths = glob("/home/alex/Projekte/TestData/Extended/Dexter/**/*.wav", recursive=True)

with open("/home/alex/Projekte/so-vits-svc/filelists/gametts_train.txt", "w") as f:
    for path in wavs_paths:
        f.write(f"{path}|0\n")